## text classification etc.

In [1]:
def find_sentiment(sentence, pos, neg):
    sentence = sentence.split()
    sentence = set(sentence)

    num_common_pos = len(sentence.intersection(pos))
    num_common_neg = len(sentence.intersection(neg))

    if num_common_pos > num_common_neg:
        return "positive"
    if num_common_pos < num_common_neg:
        return "negative"
    return neutral

In [2]:
from nltk.tokenize import word_tokenize

sentence = "hi, how are you?"
sentence.split()

['hi,', 'how', 'are', 'you?']

In [3]:
word_tokenize(sentence)

['hi', ',', 'how', 'are', 'you', '?']

In [4]:
# bag of word as initial trial

from sklearn.feature_extraction.text import  CountVectorizer

corpus = [
    "hello, how are you?",
    "im getting bored at home. And you? What do you think?", "did you know about counts",
    "let's see if this works!",
    "YES!!!!"
]
ctv = CountVectorizer()
ctv.fit(corpus)
corpus_transformed = ctv.transform(corpus)
corpus_transformed

<5x23 sparse matrix of type '<class 'numpy.int64'>'
	with 25 stored elements in Compressed Sparse Row format>

In [5]:
ctv.vocabulary_

{'hello': 9,
 'how': 11,
 'are': 2,
 'you': 22,
 'im': 13,
 'getting': 8,
 'bored': 4,
 'at': 3,
 'home': 10,
 'and': 1,
 'what': 19,
 'do': 7,
 'think': 17,
 'did': 6,
 'know': 14,
 'about': 0,
 'counts': 5,
 'let': 15,
 'see': 16,
 'if': 12,
 'this': 18,
 'works': 20,
 'yes': 21}

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import  word_tokenize

ctv = CountVectorizer(tokenizer=word_tokenize, token_pattern=None)
ctv.fit(corpus)
corpus_transformed = ctv.transform(corpus)
print(ctv.vocabulary_)

{'hello': 14, ',': 2, 'how': 16, 'are': 7, 'you': 27, '?': 4, 'im': 18, 'getting': 13, 'bored': 9, 'at': 8, 'home': 15, '.': 3, 'and': 6, 'what': 24, 'do': 12, 'think': 22, 'did': 11, 'know': 19, 'about': 5, 'counts': 10, 'let': 20, "'s": 1, 'see': 21, 'if': 17, 'this': 23, 'works': 25, '!': 0, 'yes': 26}
